In [1]:
pip install pymysql


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 932.1 kB/s eta 0:00:00


In [3]:
import pandas as pd
from sqlalchemy import create_engine

# Configuração de acesso ao banco de dados
DATABASE = 'a4f2b49a_sample_database'
HOST = '40b8f30251.nxcli.io'
USER = 'a4f2b49a_padawan'
PASSWORD = 'KaratFlanksUgliedSpinal'
PORT = 3306

# Criando uma conexão com o banco de dados
connection_string = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)
connection = engine.connect()

# Query para buscar os dados da tabela raw_data
query = "SELECT * FROM raw_data"

# Lê os dados do MySQL para um DataFrame
df = pd.read_sql(query, connection)

# Consolidação dos dados
df['mes'] = df['datahora_acesso'].dt.to_period('M')
consolidated_df = df.groupby('mes').agg({
    'rake': 'sum',
    'modalidade': 'count',
    'rake': 'sum',
    'modalidade': pd.Series.nunique,
    'rake_cash_game': lambda x: x[df['modalidade'] == 'Cash Game'].sum(),
    'rake_torneio': lambda x: x[df['modalidade'] == 'Torneio'].sum(),
    'clientes_id': pd.Series.nunique,
    'jogadores_cash_game': lambda x: x[df['modalidade'] == 'Cash Game'].nunique().sum(),
    'jogadores_torneio': lambda x: x[df['modalidade'] == 'Torneio'].nunique.sum()()
}).reset_index()

# Calcula o número de novos jogadores
first_month_players = df.groupby('clientes_id')['datahora_acesso'].min().dt.to_period('M').reset_index()
new_players = first_month_players[first_month_players['datahora_acesso'] == first_month_players['datahora_acesso'].min()]
new_players_count = new_players.shape[0]

# Adiciona a coluna de novos jogadores ao DataFrame consolidado
consolidated_df['novos_jogadores'] = new_players_count

# Salva os dados consolidados em uma nova tabela no banco de dados
consolidated_df.to_sql('consolidated_data', connection, if_exists='replace', index=False)

# Fecha a conexão com o banco de dados
connection.close()

print("Dados consolidados salvos com sucesso na tabela 'consolidated_data'.")


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '40b8f30251.nxcli.io' (timed out)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)